In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

In [2]:
import kfp
from os import environ

In [3]:
%nuclio config spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


In [27]:
%%nuclio env 
EXPERIMENT=my experiment
RUN_NAME=my run

%nuclio: setting 'EXPERIMENT' environment variable
%nuclio: setting 'RUN_NAME' environment variable


In [16]:
%%nuclio cmd -c
pip3 install https://storage.googleapis.com/ml-pipeline/release/0.1.20/kfp.tar.gz --upgrade
apt-get update && apt-get install -y wget
wget -O /tmp/pipeline.yaml https://raw.githubusercontent.com/v3io/kftools/master/examples/basic-pipeline.yaml

In [19]:
def handler(context, event):
    client = kfp.Client()
    experiment = client.create_experiment(environ.get('EXPERIMENT'))
    run_result = client.run_pipeline(experiment.id, environ.get('RUN_NAME'), '/tmp/pipeline.yaml', {})

In [22]:
# nuclio: ignore
event = nuclio.Event(body='')
handler(context, event)

In [23]:
%nuclio show

%nuclio: notebook exported to /tmp/tmpnu8tl3df/kfp-trigger.yaml
Code:
# Generated by nuclio.export.NuclioExporter on 2019-06-18 20:10

import kfp
from os import environ

def handler(context, event):
    client = kfp.Client(namespace=environ.get('KF_NAMESPACE','kubeflow'))
    experiment = client.create_experiment(environ.get('EXPERIMENT'))
    run_result = client.run_pipeline(experiment.id, environ.get('RUN_NAME'), '/tmp/pipeline.yaml', {})


Config:
apiVersion: nuclio.io/v1
kind: Function
metadata:
  name: kfp-trigger
spec:
  build:
    baseImage: python:3.6-jessie
    commands:
    - pip3 install https://storage.googleapis.com/ml-pipeline/release/0.1.20/kfp.tar.gz
      --upgrade
    - apt-get update && apt-get install -y wget
    - wget -O /tmp/pipeline.yaml https://raw.githubusercontent.com/v3io/kftools/master/examples/basic-pipeline.yaml
    noBaseImagesPull: true
  env:
  - name: KF_NAMESPACE
    value: default-tenant
  - name: EXPERIMENT
    value: my experiment
  - name: RUN_NA

In [25]:
%nuclio deploy -n pipe-init -p pipeline -d <nuclio-dashboard-url>

%nuclio: ['deploy', '-n', 'pipe-init', '-p', 'pipeline', '-c', '/User/kfp-trigger.ipynb']
%nuclio: [nuclio.deploy] 2019-06-18 20:12:21,286 project name not found created new (pipeline)
%nuclio: [nuclio.deploy] 2019-06-18 20:12:23,341 (info) Building processor image
%nuclio: [nuclio.deploy] 2019-06-18 20:13:01,623 (info) Pushing image
%nuclio: [nuclio.deploy] 2019-06-18 20:13:17,742 (info) Build complete
%nuclio: [nuclio.deploy] 2019-06-18 20:13:21,771 (info) Function deploy complete
%nuclio: [nuclio.deploy] 2019-06-18 20:13:21,775 done creating pipe-init, function address: 3.122.249.121:31186
%nuclio: function deployed


In [ ]:
# trigger the pipeline
!curl 3.122.249.121:31186